In [ ]:
import re
import pandas as pd
import numpy as np

import torch
import jionlp as jio

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
# load the model and tokenizer
device = torch.device('cuda')
tokenizer = AutoTokenizer.from_pretrained("trained_model/small_helmet_418M")
device = torch.device('cuda')
#change the model here
model = AutoModelForSeq2SeqLM.from_pretrained('trained_model/small_helmet_418M')

# 答案生成區塊

In [ ]:
def generate_answer(question):
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=256).to(device)
    out = model.generate(**inputs)
    return tokenizer.decode(out[0], skip_special_tokens=True)

model = model.to(device)

In [ ]:
#將問題輸入進去
question = ''
generate_answer(question)


# ROUGE


In [ ]:
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
for month in month_list:
    df = pd.read_csv(f'./dataset/dataset_training/2023{month}.csv')
    if month == '01':
        df_all = df
    else:
        df_all = pd.concat([df_all, df], axis=0)

data = df_all[['p_claim', 'p_fact']]
data = data.rename(columns={'p_claim': 'answer', 'p_fact': 'question'})
data = data.dropna()
data = data.reset_index(drop=True)
data['question'] = data['question'].apply(lambda x: jio.clean_text(x))
data['answer'] = data['answer'].apply(lambda x: jio.clean_text(x))
data['question'] = data['question'].apply(lambda x: re.sub(r'\s+', ' ', x))
data['answer'] = data['answer'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [ ]:
#take out the first 100 data for testing
data = data.iloc[:100]


In [ ]:
from datasets import load_metric
rouge= load_metric("rouge")
predictions = []
references = []
for i in range(len(data)):
    question = data['question'][i]
    answer = data['answer'][i]
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=256).to(device)
    out = model.generate(**inputs)
    prediction = tokenizer.decode(out[0], skip_special_tokens=True)
    rouge.compute(predictions=[prediction], references=[answer])

In [ ]:
rouge.compute(predictions=[prediction], references=[answer])